In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from kfsims.network import create_network, update_node_by_neighbors
from kfsims.common import init_trajectory
import numpy as np

In [3]:
traj = init_trajectory(None)

In [9]:
NODES = 10
ITERATIONS = 10

# No fusion

In [10]:
# no fusion
def no_fusion_step_by_step(nodes=NODES, its=ITERATIONS):
    net = create_network(nodes, iterations=its)
    msrms = {node: (i for i in node.observe()) for node in net}
    rmses = []
    for i in range(traj.X.shape[1]):
        for node, ms in msrms.items():
            m = next(ms)
            node.single_kf(m)
    for node in net:
        rmses.append(node.post_rmse(traj.X))
    return round(np.mean(rmses), 3)

In [11]:
no_fusion_step_by_step(10, 100)

6.341

In [12]:
#for i in [1, 2, 5, 7, 10, 15, 20, 25, 26, 27, 28, 30, 50, 100]:
#    for no in [7, 10, 15, 20, 25, 26, 27, 28, 30, 50, 100]:
#        try:
#            print (no, i, no_fusion_step_by_step(no, i))
#        except:
#            pass
    #no_fusion_step_by_step()

In [18]:
for i in [1, 2, 5, 7, 10, 15, 20, 25, 26, 27, 28, 30, 50, 100]:
    print (i, no_fusion_step_by_step(NODES, i))
#no_fusion_step_by_step()

1 5.329
2 4.565
5 5.216
7 4.752
10 5.365
15 5.125
20 4.261
25 5.156
26 6.323
27 5.557
28 5.697
30 4.769
50 5.708
100 5.246


In [13]:
for i in [1, 2, 5, 7, 10, 15, 20, 25, 26, 27, 28, 30, 50, 100]:
    print (i, no_fusion_step_by_step(NODES, i))
#no_fusion_step_by_step()

1 5.95
2 4.993
5 5.161
7 4.909
10 5.906
15 4.72
20 5.147
25 6.018
26 3.94
27 4.38
28 5.04
30 5.863
50 5.592
100 4.393


In [14]:
def no_fusion_all_at_once():
    # all at once, should be approx. same as the above
    # just a sanity check that the the step by step works
    # as expected since it's needed in diffusion
    net = create_network(NODES, iterations=ITERATIONS)
    rmses = []
    for node in net.nodes:
        node()
        rmses.append(node.post_rmse(traj.X))
    return np.mean(rmses)

In [15]:
no_fusion_all_at_once()

5.7932559666219685

# Simple diffusion 

In [16]:
def simple_diffusion():
    net = create_network(NODES, 5, ITERATIONS)
    msrms = {node: (i for i in node.observe()) for node in net}
    rmses = []
    for i in range(traj.X.shape[1]):

        # proveď jeden KF loop pro každý node
        for node, ms in msrms.items():
            m = next(ms)
            node.single_kf(m)

        # každému nodu přiřaď průměr hyperparametrů od všech sousedů
        for node in net:
            update_node_by_neighbors(net, node)


    for node in net:
        rmses.append(node.post_rmse(traj.X))
    return np.mean(rmses)

In [17]:
simple_diffusion()

6.08748947967541